In [ ]:
pip install arch

In [ ]:
pip install riskfolio-lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 23.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import arch
from arch import arch_model
from riskfolio import VaR_Hist
from riskfolio import CVaR_Hist
import seaborn as sns

In [ ]:
ret = pd.read_excel(r'return.xlsx')
ret.set_index('Date', inplace= True)

FileNotFoundError: ignored

In [ ]:
#returns
#returns['Date'] = returns['Date'].astype('datetime64')
#returns.set_index('Date', inplace= True)

In [ ]:
ret

,vhm,vcb,fpt,agg,agr,asp,msb,nvl,ree
Date,,,,,,,,,
2020-12-24,-0.012665,-0.002054,-0.008734,-0.045068,-0.043616,-0.034412,0.017492,-0.015040,-0.016632
2020-12-25,0.014951,0.003079,0.010471,0.036556,0.047067,0.032790,0.067059,0.007550,0.006271
2020-12-28,0.001141,0.004090,0.005196,-0.015505,0.000000,-0.016252,0.013429,0.008979,-0.007321
2020-12-29,0.029215,0.012170,0.012017,-0.008716,-0.017387,0.006529,0.013243,0.011854,0.001049
2020-12-30,-0.001108,-0.013191,-0.008569,-0.010564,-0.018893,0.001621,-0.005280,-0.016333,0.012501
...,...,...,...,...,...,...,...,...,...
2023-03-27,-0.012320,0.022223,0.006341,0.000000,0.042966,0.005479,0.000000,0.065064,0.007380
2023-03-28,-0.008299,0.008753,0.001263,-0.001771,-0.007034,-0.001823,0.004107,-0.007905,0.000000
2023-03-29,0.000000,0.005432,-0.002528,-0.003552,0.022107,0.000000,0.000000,-0.003976,-0.011834


In [ ]:
ret.describe()

,vhm,vcb,fpt,agg,agr,asp,msb,nvl,ree
count,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000
mean,-0.000397,0.000339,0.001210,0.000385,0.000170,-0.000117,0.000399,-0.001125,0.000857
std,0.021049,0.018322,0.019052,0.018938,0.034247,0.029956,0.023783,0.028062,0.025143
min,-0.072425,-0.072372,-0.072400,-0.097896,-0.077782,-0.112869,-0.072404,-0.072546,-0.072539
25%,-0.011177,-0.010132,-0.006542,-0.006951,-0.018229,-0.013979,-0.009394,-0.009975,-0.012028
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001247
75%,0.009593,0.009792,0.009695,0.006419,0.018302,0.013178,0.010809,0.009039,0.011904
max,0.067467,0.060625,0.067628,0.096159,0.067661,0.067663,0.067659,0.207897,0.067471


In [ ]:
train_data = ret.iloc[:500,:]
test_data = ret.iloc[500:,:]

In [ ]:
coeffs = []
cond_vol = []
std_resids = []
models = []
infor_aic = []
infor_bic = []
pred = []

In [ ]:
for asset in train_data.columns:
    model = arch_model(train_data[asset], mean='zero', vol='GARCH', p=1, o=1, q=1).fit(update_freq=0, disp='off')
    predict = model.forecast(horizon= len(test_data), method= 'analytic')
    coeffs.append(model.params)
    cond_vol.append(model.conditional_volatility)
    std_resids.append(model.resid / model.conditional_volatility)
    models.append(model)
    infor_aic.append(model.aic)
    infor_bic.append(model.bic)
    pred.append(predict.variance.values[-1])

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004252. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\arch\univariate\bas

In [ ]:
models

[                     Zero Mean - GJR-GARCH Model Results                      
 Dep. Variable:                    vhm   R-squared:                       0.000
 Mean Model:                 Zero Mean   Adj. R-squared:                  0.002
 Vol Model:                  GJR-GARCH   Log-Likelihood:                1263.51
 Distribution:                  Normal   AIC:                          -2519.01
 Method:            Maximum Likelihood   BIC:                          -2502.16
                                         No. Observations:                  500
 Date:                Mon, Apr 24 2023   Df Residuals:                      500
 Time:                        00:44:46   Df Model:                            0
                               Volatility Model                              
                  coef    std err          t      P>|t|      95.0% Conf. Int.
 ----------------------------------------------------------------------------
 omega      4.4016e-05  6.214e-06      7.084  

In [ ]:
coeffs_df = pd.DataFrame(coeffs).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
cond_vol_df = pd.DataFrame(cond_vol).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
std_resids_df = pd.DataFrame(std_resids).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
pred_df = pd.DataFrame(np.sqrt(pred)).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
pred_df.set_index(test_data.index, inplace= True)
MSE = np.mean((pred_df - test_data)**2)
infor_df = pd.DataFrame({'AIC': infor_aic, 'BIC': infor_bic, 'MSE': MSE}).set_index(train_data.columns, inplace=False)

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [ ]:
coeffs_df

,vhm,vcb,fpt,agg,agr,asp,msb,nvl,ree
omega,0.000044,0.000102,0.000008,0.000008,0.000177,0.000247,0.000178,0.000020,0.000068
alpha[1],0.118062,0.100000,0.100000,0.050000,0.105683,0.214871,0.010000,0.083189,0.010000
gamma[1],0.032962,0.050000,0.010000,0.010000,0.090928,-0.046232,0.200000,0.151270,0.200000
beta[1],0.759753,0.575000,0.875000,0.925000,0.705711,0.553166,0.590000,0.851131,0.790000


In [ ]:
coeffs_df.to_excel('TGARCH.xlsx', index=False)

In [ ]:
infor_df

,AIC,BIC,MSE
vhm,-2519.013887,-2502.155455,0.000979
vcb,-2592.893375,-2576.034942,0.000560
fpt,-2566.678165,-2549.819733,0.000392
agg,-2532.520146,-2515.661714,0.000540
agr,-1955.204252,-1938.345819,0.001659
asp,-2081.148892,-2064.290460,0.001144
msb,-2324.043502,-2307.185070,0.000927
nvl,-2400.874659,-2384.016227,0.007359
ree,-2298.657620,-2281.799187,0.000951


## Giá trị VaR và CVaR dự báo

In [ ]:
VaR_95 = []
CVaR_95 = []
VaR_99 = []
CVaR_99 = []

for i in pred_df.columns:
    cal_VaR_95 = VaR_Hist(pred_df[i], alpha= 0.05)
    cal_CVaR_95 = CVaR_Hist(pred_df[i], alpha= 0.05)
    cal_VaR_99 = VaR_Hist(pred_df[i], alpha= 0.01)
    cal_CVaR_99 = CVaR_Hist(pred_df[i], alpha= 0.01)
    VaR_95.append(cal_VaR_95)
    CVaR_95.append(cal_CVaR_95)
    VaR_99.append(cal_VaR_99)
    CVaR_99.append(cal_VaR_99)

VaR_df = pd.DataFrame({'VaR 95%': VaR_95,
                       'VaR 99%': VaR_99,
                       'CVaR 95%': CVaR_95,
                       'CVaR 99%': CVaR_99}).set_axis(train_data.columns, inplace=False)
VaR_df

,VaR 95%,VaR 99%,CVaR 95%,CVaR 99%
vhm,-0.020408,-0.020407,-0.020407,-0.020407
vcb,-0.017521,-0.015691,-0.016524,-0.015691
fpt,-0.014780,-0.014399,-0.014544,-0.014399
agg,-0.020115,-0.020099,-0.020105,-0.020099
agr,-0.034855,-0.034670,-0.034745,-0.034670
asp,-0.029163,-0.026136,-0.027476,-0.026136
msb,-0.023815,-0.022720,-0.023214,-0.022720
nvl,-0.060758,-0.059367,-0.059888,-0.059367
ree,-0.024046,-0.023209,-0.023543,-0.023209


In [ ]:
# Tính trọng số đều cho các mã
weights = np.array([1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9])

# Tính VaR và CVaR của danh mục đầu tư
portfolio_VaR_95 = np.sum(weights * VaR_95)
portfolio_VaR_99 = np.sum(weights * VaR_99)
portfolio_CVaR_95 = np.sum(weights * CVaR_95)
portfolio_CVaR_99 = np.sum(weights * CVaR_99)

risk = pd.DataFrame(data= [portfolio_VaR_95, portfolio_VaR_99, portfolio_CVaR_95, portfolio_CVaR_99],
                    index= ['VaR 95%', 'VaR 99%', 'CVaR 95%', 'CVaR 99%'], columns= ['Portfolio'])
risk

,Portfolio
VaR 95%,-0.027273
VaR 99%,-0.026300
CVaR 95%,-0.026716
CVaR 99%,-0.026300


In [ ]:
#from scipy.optimize import minimize

In [ ]:
#def objective_function(weights, VaR_99, CVaR_99):
   # portfolio_VaR = np.sum(weights * VaR_99)
   # portfolio_CVaR = np.sum(weights * CVaR_99)
   # return portfolio_CVaR - portfolio_VaR

In [ ]:
# Thêm ràng buộc và hạn chế để đảm bảo rằng tổng trọng số bằng 1 và các trọng số không âm
#def constraint1(weights):
    #return np.sum(weights) - 1

In [ ]:
#constraints = [{'type': 'eq', 'fun': constraint1}]
#bounds = [(0, 1)] * len(VaR)

In [ ]:
#optimized = minimize(objective_function,
                   #  x0 = [1.0/len(train_data.columns)]*len(train_data.columns),
                   #  args=(VaR_95, CVaR_95),
                   #  method='SLSQP',
                   #  bounds=bounds,
                   #  constraints=constraints)

In [ ]:
#new_weight = pd.DataFrame(optimized.x, columns= ['weight']).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
#new_weight.applymap('{:.2%}'.format)